In [0]:
!pip install pycld2
!pip install -U git+https://github.com/aboSamoor/polyglot.git@master

!pip install fasttext
!wget -O /tmp/lid.176.bin https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin

     |████████████████████████████████| 41.4MB 110kB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp27-cp27mu-linux_x86_64.whl size=9831801 sha256=5762c1f98822ceec62a65e84802c20313dfc8543582aee45d5ff3872f71770ed
  Stored in directory: /root/.cache/pip/wheels/c6/8f/e9/08a1a8932a490175bd140206cd86a3dbcfc70498100de11079
Successfully built pycld2
  Cloning https://github.com/aboSamoor/polyglot.git (to revision master) to /tmp/pip-req-build-7qwue4
  Running command git clone -q https://github.com/aboSamoor/polyglot.git /tmp/pip-req-build-7qwue4
     |████████████████████████████████| 225kB 2.7MB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=70644 sha256=cc983eab2b59a9ba8d8f629e59f555384f7c6b11ea0648c43ffc9a19e3736145
  Stored in directory: /tmp/pip-ephem-wheel-cache-Kwf4bC/wheels/42/d9/73/345c7ae8554299ff8b31635d64eb8455fd591385fa734cdbef
  Created wheel for PyICU: filename=PyICU-2.4.3-cp27-cp27mu-linux_x86_64.whl size=872618 sha256=71d339aba3a5c

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from __future__ import print_function, division

from polyglot.detect import Detector
from ftfy import fix_encoding
from collections import Counter

import pandas as pd
import numpy as np

import re
import csv
import fasttext

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)

hashtag_pattern = re.compile(u"#\w+")

userhandle_pattern = re.compile(u"@(\w){1,30}")

def fix_text(text):
  # Emoji remover
  text = emoji_pattern.sub(r'', text)
  # Hashtag remover
  text = hashtag_pattern.sub(r'', text)
  # Userhandle remover
  text = userhandle_pattern.sub(r'', text)
  # Link remover
  text = re.sub(r"http\S+", r'', text)
  # Punctation remover
  text = re.sub(ur'[.,\/#!?$%\^&\*;:{}=\-_`~()]', r'', text)
  # New line to space character
  text = re.sub(r"[\r\n]", " ", text)
  # Reduce multiple space characters to only one
  text = re.sub(r"\s\s+", " ", text)
  # Lowercase all characters
  text = text.lower()

  return text

def label_lang_polyglot(text):
  try:
    lang = Detector(text, quiet=True).language.code
  except:
    lang = "err"
  return lang

def label_lang_fasttext(text):
  try:
    lang = str(model.predict(text)[0][0]).split('__')[2]
  except:
    lang = "err"
  return lang

PRETRAINED_MODEL_PATH = '/tmp/lid.176.bin'
model = fasttext.load_model(PRETRAINED_MODEL_PATH)

def extract_majority_vote(text):
  languages = text.split("-")

  i = 0
  while i < len(languages):
    if languages[i] == 'un':
      languages[i] = 'und'
    i = i + 1

  languages = Counter(languages)
  max_count = 0
  max_lang = "conflict"
  for lang in languages:
    if languages[lang] > len(languages)/2:
      max_lang = lang
      break

  return max_lang

LANG_CODES = ["BCL", "CBK" ,"CEB", "EN", "ILO", "PAM", "TL", "WAR", "UN", "UND", "conflict"]

def extract_language_counts(s):
  lang_dict = {}
  _sum = 0
  for code in LANG_CODES:
    lang_dict[code.lower()] = len(s[s == code.lower()])
    _sum = _sum + len(s[s == code.lower()])
  lang_dict['other'] = len(s) - _sum
  return lang_dict

In [0]:
# FILE PATHS REMOVED FROM COLAB
INSTAGRAM_POSTS_FILE_PATH
TWITTER_TWEETS_FILE_PATH
INSTAGRAM_LANG_TAGS_FILE_PATH
TWITTER_LANG_TAGS_FILE_PATH

In [0]:
print("Loading Instagram posts...")
instagram_df = pd.read_csv(
	INSTAGRAM_POSTS_FILE_PATH, 
	encoding="utf-8",
	quoting=csv.QUOTE_ALL)
instagram_df.set_index("InstaPostID", inplace=True)
instagram_df = instagram_df[['UserID', 'Caption']]

print("Loading Twitter posts...")
twitter_df = pd.read_csv(
	TWITTER_TWEETS_FILE_PATH, 
	encoding="utf-8",
	quoting=csv.QUOTE_ALL)
twitter_df.set_index("TweetID", inplace=True)
twitter_df = twitter_df[['UserID', 'Text', 'Language']]
twitter_df['Text'] = twitter_df['Text'].fillna("")

Loading Instagram posts...
Loading Twitter posts...


In [0]:
instagram_df.head()

In [0]:
twitter_df.head()

In [0]:
print("INITIAL STATISTICS:")
print("TOTAL TWITTER POSTS: %d" % len(twitter_df))
print("TOTAL INSTAGRAM POSTS: %d" % len(instagram_df))
print("TOTAL INSTAGRAM POSTS W/O TEXT: %d" % len(instagram_df[instagram_df.Caption.isna()]))

INITIAL STATISTICS:
TOTAL TWITTER POSTS: 4018628
TOTAL INSTAGRAM POSTS: 195757
TOTAL INSTAGRAM POSTS W/O TEXT: 17107


In [0]:
print("Removing captions w/ no text...")
instagram_df = instagram_df[~instagram_df.Caption.isna()]
print("done.")

Removing captions w/ no text...
done.


In [0]:
print("Fixing text...")
instagram_df['Caption'] = instagram_df['Caption'].apply(fix_text)
print("Instagram done.")
twitter_df['Text'] = twitter_df['Text'].apply(fix_text)
print("Twitter done.")

Fixing text...
Instagram done.
Twitter done.


In [0]:
instagram_df.head()

In [0]:
twitter_df.head()

In [0]:
print("Detecting language polyglot...")
instagram_df['Lang_Polyglot'] = instagram_df['Caption'].apply(label_lang_polyglot)
twitter_df['Lang_Polyglot'] = twitter_df['Text'].apply(label_lang_polyglot)

print("Detecting language fasttext...")
instagram_df['Lang_FastText'] = instagram_df['Caption'].apply(label_lang_fasttext)
twitter_df['Lang_FastText'] = twitter_df['Text'].apply(label_lang_fasttext)

Detecting language polyglot...
Detecting language fasttext...


In [0]:
print("Extracting results (single label)...")
instagram_df['LangID_Combined'] = instagram_df['Lang_Polyglot'] + "-" + instagram_df['Lang_FastText']
twitter_df['LangID_Combined'] = twitter_df['Language'] + "-" + twitter_df['Lang_Polyglot'] + "-" + twitter_df['Lang_FastText']

Extracting results (single label)...


In [0]:
instagram_df['MajorityVote'] = instagram_df['LangID_Combined'].apply(extract_majority_vote)
twitter_df['MajorityVote'] = twitter_df['LangID_Combined'].apply(extract_majority_vote)

In [0]:
print("Saving files...")
instagram_df.to_csv("InstaPostsWLanguage.csv", encoding='utf-8', quoting=csv.QUOTE_ALL)
twitter_df.to_csv("TwitaPostsWLanguage.csv", encoding='utf-8', quoting=csv.QUOTE_ALL)

Saving files...


#ALL SCRIPTS FOR TOTAL LANGAUGE DISTRIBUTION

In [0]:
all_languages = {
    'Twitter Tag': extract_language_counts(twitter_df['Language']),
    'Polyglot': extract_language_counts(twitter_df['Lang_Polyglot']),
    'FastText': extract_language_counts(twitter_df['Lang_FastText']),
    'Majority Vote': extract_language_counts(twitter_df['MajorityVote']),
}

df_tweet_lang_table = pd.DataFrame.from_dict(all_languages)
print(df_tweet_lang_table)

          FastText  Majority Vote  Polyglot  Twitter Tag
bcl            358              0         0            0
cbk            479              0         0            0
ceb         115678          34128    121877            0
conflict         0         661954         0            0
en         2244161        1855462   2094950      1563939
ilo           8605              0         0            0
other       685386          41862    540371       245900
pam            634              0         0            0
tl          933657        1264173   1040594      1945342
un               0              0    171582            0
und              0         159338         0       263447
war          29670           1711     49254            0


In [0]:
for code in LANG_CODES:
  print(code, twitter_df['Text'][twitter_df['Lang_FastText'] == code.lower()][:10])
  print("==========")

In [0]:
all_languages = {
    'Polyglot': extract_language_counts(instagram_df['Lang_Polyglot']),
    'FastText': extract_language_counts(instagram_df['Lang_FastText']),
    'Majority Vote': extract_language_counts(instagram_df['MajorityVote']),
}

df_ig_lang_table = pd.DataFrame.from_dict(all_languages)
print(df_ig_lang_table)

          FastText  Majority Vote  Polyglot
bcl              7              0         0
cbk              9              0         0
ceb           1329            192       781
conflict         0          46476         0
en          147892         123962    135450
ilo             73              0         0
other        19455            918     13609
pam              6              0         0
tl            9654           7089     10966
un               0              0     17191
und              0              0         0
war            225             13       653


In [0]:
!cp TwitaPostsWLanguage.csv TWITTETR_LANG_TAGS_FILE_PATH
!cp InstaPostsWLanguage.csv INSTAGRAM_LANG_TAGS_FILE_PATH